In [ ]:
# Get all the information for each publication for each grant to copy to the spreadsheets

In [ ]:
import os
import json
import collections

pubs_grant_primary = collections.defaultdict(dict)
pubs_grant_secondary = collections.defaultdict(dict)
pubs_grant_secondary_spreadsheet = collections.defaultdict(dict)


fname_grant_jsons = os.listdir("/Users/luisa/pub_tracker_test/data_grant")
x = 0
for fname_json in fname_grant_jsons:
    with open("/Users/luisa/pub_tracker_test/data_grant/" + fname_json,"r") as fp:
        x = x + 1
        dic_pergrant = json.load(fp)
        pubs = dic_pergrant.keys()
        grant_name = fname_json[:-5]
        print(grant_name)
        for pub in pubs:
            authors = {pub: "; ".join(dic_pergrant[pub]["authors"]) for pub in pubs}
            title = {pub:dic_pergrant[pub]["title"] for pub in pubs}
            link = {pub:dic_pergrant[pub]["link"] for pub in pubs}
            date = {pub:dic_pergrant[pub]["date"] for pub in pubs}
            other_ids = {pub:";".join(dic_pergrant[pub]["other_ids"]) for pub in pubs}
            primary_awards = {pub:";".join(dic_pergrant[pub]["primary_awards"]) if len(dic_pergrant[pub]["primary_awards"]) > 1 else " -- " for pub in pubs}
            secondary_awards = {pub:";".join(dic_pergrant[pub]["secondary_awards"]) for pub in pubs}
            contact_4DN_author = {pub:dic_pergrant[pub]["4DN_member_author"] for pub in pubs}
            added_date = {pub:dic_pergrant[pub]["added"] for pub in pubs}
            
    pubs_grant_primary[grant_name]['title']= [title[pub] for pub in title]
    pubs_grant_primary[grant_name]['authors']= [authors[pub] for pub in authors]
    pubs_grant_primary[grant_name]['date']= [date[pub] for pub in date]
    pubs_grant_primary[grant_name]['link']= [link[pub] for pub in link]
    pubs_grant_primary[grant_name]['other_ids'] = [other_ids[pub] for pub in other_ids]
    pubs_grant_primary[grant_name]['primary_awards'] = [primary_awards[pub] for pub in primary_awards]
    pubs_grant_primary[grant_name]['secondary_awards'] = [secondary_awards[pub] for pub in secondary_awards]
    pubs_grant_primary[grant_name]['date_added'] = [added_date[pub] for pub in added_date]
    pubs_grant_primary[grant_name]['contact_4DN_author'] = [contact_4DN_author[pub] for pub in contact_4DN_author]
    
    #print(pubs_grant_primary[grant_name])
    for key in secondary_awards:
        #print(key)
        if secondary_awards[key] != '':
            if ';' not in secondary_awards[key]:
                pubs_grant_secondary[secondary_awards[key]][key] = dic_pergrant[key]
            else:
                sec_awards_list = i.split(';')
                for sec_award in sec_awards_list:
                        pubs_grant_secondary[sec_award][key] = dic_pergrant[key]

for grant in pubs_grant_secondary:
    pubs_grant_secondary_spreadsheet[grant]['title'] = [pubs_grant_secondary[grant][pub]["title"] for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['authors'] =  ["; ".join(pubs_grant_secondary[grant][pub]["authors"]) for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['date'] = [pubs_grant_secondary[grant][pub]["date"] for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['link'] = [pubs_grant_secondary[grant][pub]["link"] for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['other_ids'] = [";".join(pubs_grant_secondary[grant][pub]["other_ids"]) for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['primary_awards'] = [ ";".join(pubs_grant_secondary[grant][pub]["primary_awards"]) for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['secondary_awards'] = [ ";".join(pubs_grant_secondary[grant][pub]["secondary_awards"]) if len(pubs_grant_secondary[grant]) > 1 else "--" for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]["added_date"] = [pubs_grant_secondary[grant][pub]["added"] for pub in pubs_grant_secondary[grant]]
    pubs_grant_secondary_spreadsheet[grant]['contact_4DN_author'] = [pubs_grant_secondary[grant][pub]["4DN_member_author"] for pub in pubs_grant_secondary[grant]]

#print(pubs_grant_secondary_spreadsheet.keys())
    

In [1]:
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

google_management_sheet_id = 'here goes the link'

In [ ]:
#Opens google management spreadsheet to read links of grant spreadhseet
google_management_spreadsheet = gc.open_by_key(google_management_sheet_id)
google_management_sheet = google_management_spreadsheet.sheet1
#print(google_management_sheet)

In [ ]:
#values_list_row = google_management_sheet.row_values(2)# Give the row number and get the values of that row
#print(values_list_row)
#values_list_col = google_management_sheet.col_values(2) # Give the col number and get the values of that col
#print(values_list_col)
#list_of_lists = worksheet.get_all_values() #Get all the values of the spreadsheet

In [ ]:
grant_sheets_links = google_management_sheet.get_all_values()

In [ ]:
print(grant_sheets_links[2])

In [ ]:
import time

col_labels = ['Title', 'Authors','Date', 'Link', 'Other ids', 'Primary_awards', 'Secondary Awards','Date added', 'Contact 4DN Author']
reqst_ct = 1
row = 2
col = 1
for grant in pubs_grant_primary:
    if grant == "U01CA200060":
        continue
    gs_write = []
    for item in grant_sheets_links:
        if grant in item[1]:
            spreadsheet_id = item[2]
            if reqst_ct >= 90:
                time.sleep(100)
                reqst_ct = 0
                spreadsheet = gc.open_by_key(spreadsheet_id)
                print('Working on ', spreadsheet)
                reqst_ct += 1
            else:
                spreadsheet = gc.open_by_key(spreadsheet_id)
                print('Working on ', spreadsheet)
                reqst_ct += 1
                
            if reqst_ct >= 90:
                time.sleep(100)
                reqst_ct = 0
                try:
                    worksheet = spreadsheet.worksheet('Publication List')   
                    reqst_ct += 1
                except:
                    worksheet = spreadsheet.add_worksheet(title='Publication List', rows='200', cols='15')
                    reqst_ct += 1
            else:
                try:
                    worksheet = spreadsheet.worksheet('Publication List')   
                    reqst_ct += 1
                            
                except:
                    worksheet = spreadsheet.add_worksheet(title='Publication List', rows='200', cols='15')
                    reqst_ct += 1
    col = 1
    row = 1
    for label in col_labels:
            gs_write.append(gspread.models.Cell(1,col,label))
            col += 1
    
    col = 1
    for item in pubs_grant_primary[grant].values():
        row = 2
        for info in item:
            gs_write.append(gspread.models.Cell(row,col,info))
            row += 1
        col += 1 

    row += 2
    new_row = row
    col = 1

    for item in pubs_grant_secondary_spreadsheet[grant].values():
        new_row = row
        for info in item:
            gs_write.append(gspread.models.Cell(row,col,info))
            new_row += 1
        col += 1 
        
        
    print(gs_write)
    print()
    worksheet.update_cells(gs_write)
    reqst_ct += 1

In [ ]:
gs_write

In [ ]:
print(pubs_grant_primary.keys())

In [ ]:
print(grant_sheets_links)

In [ ]:
# Opens json that lists the contact PI for each grant
#with open("/Users/luisa/pub_tracker_test/4DN_official_awards.json", 'r') as file:
#    contact_pi_grants = json.load(file)

In [ ]:
#print(contact_pi_grants['TCPA-2017-09'])

In [ ]:
#grants_4dn = contact_pi_grants.keys()

In [ ]:
#print(len(grants_4dn))

In [ ]:
#creates a cell object
#data = gspread.models.Cell(1,1,'Hello')

In [ ]:
with open('/Users/luisa/pub_tracker_test/data_post/biorxiv-024620.json', 'r') as f:
    data = json.load(f)

In [ ]:
print(data)

In [ ]:
latest = data['latest']

In [ ]:
print(latest)

In [ ]:
latest['awards']

In [ ]:
len(latest['awards'])

In [ ]:
latest['awards'][0] == 'NONE'